# Merging data frames and cleaning tweets data

In [1]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import date, timedelta


In [48]:
# Display all the text in columns
pd.set_option('display.max_colwidth',None)

## Elon Musks Tweet Data
#### Cleaning data frame for NLP 

In [2]:
# Import csv file with tweets for elon musk
file_name='elon_tweets.csv'
tweets_df = pd.read_csv(Path(f"Resources/{file_name}"),parse_dates=True, infer_datetime_format=True) # we do not do date as index just yet

# we change the name to date column - we will drop this field later. We need a date field that shows off market hour tweets as t+1 
tweets_df.rename(columns={'date':'date original'},inplace=True)
tweets_df['date original']=pd.to_datetime(tweets_df['date original'])

# Make tweets made after market hours fall into the following day
# Define market hour limit as everything after 16hs 00 min 00 sec
min_hour=16
min_minute=0
min_second=0

# we create the new field equalt to date original 
tweets_df['date']=tweets_df['date original'].copy()

# we add 1 day to date original if the tweet occured off market hours
tweets_df.loc[(tweets_df['date original'].dt.hour>=min_hour) & (tweets_df['date original'].dt.minute>min_minute) & (tweets_df['date original'].dt.second>min_second), 'date'] = tweets_df['date original']+timedelta(days=1)

# Drop original date and make the new date column as index
tweets_df.drop(columns={'date original'}, inplace=True)
tweets_df.set_index('date', inplace=True)

# We eliminate the seconds from Tweets data frame
tweets_df.index = tweets_df.index.date


# Cleaning tweets
# Tweets made during weekends and holidays;
# Multiple tweets per day 
new_tweets_df = tweets_df.groupby(tweets_df.index).agg(' '.join).sort_index()
new_tweets_df['tweet count']=tweets_df.groupby(tweets_df.index).count().sort_index()

In [3]:
new_tweets_df.head()

,tweet,tweet count
2010-06-04,"Please ignore prior tweets, as that was someon...",1
2011-12-01,I made the volume on the Model S http://t.co/...,2
2011-12-03,That was a total non sequitur btw Great Voltai...,2
2011-12-04,Am reading a great biography of Ben Franklin b...,1
2011-12-21,"Yeah, this really is me, as my Mom @mayemusk w...",6


## Tesla and QQQ Stock Price Data
### Cleaning 

In [4]:
file_name='stock_price.csv'
stock_price_df = pd.read_csv(Path(f"Resources/{file_name}"),index_col='date',parse_dates=True, infer_datetime_format=True)

# We do the same format change to data  to make sure that the fields are comparable
stock_price_df.index = stock_price_df.index.date
stock_price_df.head()

,TSLA,QQQ
2010-06-29,23.94,43.40
2010-06-30,23.63,39.16
2010-07-01,21.90,42.58
2010-07-02,19.20,38.94
2010-07-06,15.98,42.60


In [91]:
clean_df.loc['2011-12-01':'2011-12-06'].head(6)

,TSLA,QQQ,tweet,tweet count,count holidays
2011-12-01,32.60,52.57,"I made the volume on the Model S http://t.co/wMCnT53M go to 11. Now I just need to work in a miniature Stonehenge... Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) http://t.co/rBQXJ9IT",2.0,1.0
2011-12-02,33.34,52.42,NaN,NaN,1.0
2011-12-03,NaN,NaN,"That was a total non sequitur btw Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.",2.0,NaN
2011-12-04,NaN,NaN,Am reading a great biography of Ben Franklin by Isaacson. Highly recommended.,1.0,NaN
2011-12-05,34.38,53.00,NaN,NaN,1.0
2011-12-06,34.88,57.08,NaN,NaN,1.0


## Merging Dataframes
 * Stock Data + Raw Tweets 

In [179]:
# Join data frames. Outer is used to not leave any data point behind. 
 
clean_df=stock_price_df.join(new_tweets_df,how='outer')
clean_df.index = pd.to_datetime(clean_df.index)
clean_df=clean_df.loc['2010-06-29':]
# clean_df.loc['2011-12-01':'2011-12-05'].head()

# count how many weekend days and holidays in a row there are - this will inform how many iterations of the weekend cleanup are needed
# clean_df['count holidays'] = clean_df.groupby(clean_df.index)['TSLA'].isnull().rank(axis=0,method='first')
# clean_df['count nulls']=np.where(clean_df['TSLA'].isnull(),1,0)
# clean_df['count holidays'] = clean_df['count nulls'].groupby(clean_df['count nulls']).transform('sum').astype(int)

# clean_df['count holidays'] = clean_df.TSLA.groupby([clean_df['TSLA']]).transform('sum')#.astype(int)
# print (df)

# df['rank_seller_by_close_date'] = df.groupby('seller_name')['close_date'].rank(method='first')

# clean_df.head()
clean_df.loc['2011-12-01':'2011-12-06'].head(6)

,TSLA,QQQ,tweet,tweet count
2011-12-01,32.60,52.57,"I made the volume on the Model S http://t.co/wMCnT53M go to 11. Now I just need to work in a miniature Stonehenge... Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) http://t.co/rBQXJ9IT",2.0
2011-12-02,33.34,52.42,NaN,NaN
2011-12-03,NaN,NaN,"That was a total non sequitur btw Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.",2.0
2011-12-04,NaN,NaN,Am reading a great biography of Ben Franklin by Isaacson. Highly recommended.,1.0
2011-12-05,34.38,53.00,NaN,NaN
2011-12-06,34.88,57.08,NaN,NaN


In [66]:
# clean_df['weekend corrected']=clean_df['tweet'].shift(1)
# clean_df['weekend corrected1']=clean_df['tweet count'].shift(1)

# # clean_df.loc[clean_df['TSLA'].isnull()].head()
# clean_df.loc[clean_df['TSLA'].notnull(),['weekend corrected','weekend corrected1']] = ['','']

# clean_df.loc['2011-12-01':'2011-12-05'].head()

In [183]:
# # If tweets where made during weekends, push to next business day
# clean_df_next1=clean_df.shift(1)
# clean_df_next2=clean_df.shift(2)
# # weekend_corected = pd.DataFrame(index=clean_df.index,columns='clean_df.columns')
# # weekend_corected = pd.DataFrame(index=clean_df.index,columns=['tx'])
# weekend_corected=clean_df.copy()


# # weekend_corected.loc[(clean_df['TSLA'].isnull()) & (clean_df_next1['TSLA'].notnull()) & (clean_df['tweet'].notnull()),['tx']]=clean_df_next1['tweet']
# # np.where((clean_df['TSLA'].isnull()) & (clean_df_next1['TSLA'].notnull()), clean_df1=clean_df.join(clean_df_next1,how='outer'))

# # df_out[(df>.5) & (df_prev >0)] = df_prev


In [182]:
# weekend fix - tweets
clean_df['NO_NaN']=np.where(clean_df['TSLA'].notnull() & clean_df['tweet'].notnull(),clean_df['tweet'],'')
clean_df['NaN1']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet'].notnull().shift(1),clean_df['tweet'].shift(1),'')
clean_df['NaN2']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet'].notnull().shift(1),clean_df['NaN1'].shift(1),'')
clean_df['NaN3']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet'].notnull().shift(1),clean_df['NaN2'].shift(1),'')
clean_df['NaN4']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet'].notnull().shift(1),clean_df['NaN3'].shift(1),'')

clean_df['tweet clean']= \
    clean_df['NO_NaN'][clean_df['TSLA'].notnull()] + ' ' + \
    clean_df['NaN1'][clean_df['TSLA'].notnull()] + ' ' + \
    clean_df['NaN2'][clean_df['TSLA'].notnull()] + ' ' + \
    clean_df['NaN3'][clean_df['TSLA'].notnull()] + ' ' + \
    clean_df['NaN3'][clean_df['TSLA'].notnull()] 

# Drop intermidiate columns created
clean_df.drop(columns=['NO_NaN','NaN1','NaN2','NaN3','NaN4'],inplace=True)


# weekend fix - tweet count; create intermidiate fields that move tweet count fields to the closest next weekday 
clean_df['NO_NaN']=np.where(clean_df['TSLA'].notnull() & clean_df['tweet count'].notnull(),clean_df['tweet count'],0)
clean_df['NaN1']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet count'].notnull().shift(1),clean_df['tweet count'].shift(1),0)
clean_df['NaN2']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet count'].notnull().shift(1),clean_df['NaN1'].shift(1),0)
clean_df['NaN3']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet count'].notnull().shift(1),clean_df['NaN2'].shift(1),0)
# not needed - in case there are 4 non market days in a row
clean_df['NaN4']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['tweet count'].notnull().shift(1),clean_df['NaN3'].shift(1),0)

# summ accross intermidiate fields
clean_df['tweet count clean']= \
    clean_df['NO_NaN'][clean_df['TSLA'].notnull()].astype(int) + \
    clean_df['NaN1'][clean_df['TSLA'].notnull()].astype(int) + \
    clean_df['NaN2'][clean_df['TSLA'].notnull()].astype(int) + \
    clean_df['NaN3'][clean_df['TSLA'].notnull()].astype(int) + \
    clean_df['NaN4'][clean_df['TSLA'].notnull()].astype(int)

# Drop intermidiate columns created
clean_df.drop(columns=['NO_NaN','NaN1','NaN2','NaN3','NaN4'],inplace=True)

# clean_df[['tweet count', 'tweet count clean','NO_NaN','NaN1','NaN2','NaN3']].sum()

# clean_df['tweet count'][clean_df['TSLA'].isnull()&clean_df['tweet count clean'].isnull()].sum()
# head(1000)
# clean_df.loc['2011-12-01':'2011-12-06'].head(6)
# clean_df.loc[clean_df['NaN4']>1].head()

In [67]:
clean_df['test0']=np.where(clean_df['TSLA'].notnull() & clean_df['tweet'].notnull(),clean_df['tweet'],'')

# weekend fix
clean_df['test1']=np.where(clean_df['TSLA'].shift(1).isnull() & clean_df['TSLA'].notnull() & clean_df['tweet'].notnull().shift(1),clean_df['tweet'].shift(1),0)
# clean_df['test2']=np.where(clean_df['TSLA'].shift(2).isnull() & clean_df['TSLA'].shift(1).isnull() & clean_df['TSLA'].notnull() & clean_df['tweet'].notnull().shift(2),(clean_df['test1'].astype(str)+' '+clean_df['tweet'].shift(2)),0)
# clean_df['test2']=np.where(clean_df['TSLA'].shift(2).isnull() & clean_df['TSLA'].notnull() & clean_df['tweet'].notnull().shift(2),(clean_df['test1'].astype(str)+' '+clean_df['tweet'].shift(2)),0)


clean_df.loc['2011-12-01':'2011-12-06'].head(6)

,TSLA,QQQ,tweet,tweet count,test0,test1
2011-12-01,32.60,52.57,"I made the volume on the Model S http://t.co/wMCnT53M go to 11. Now I just need to work in a miniature Stonehenge... Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) http://t.co/rBQXJ9IT",2.0,"I made the volume on the Model S http://t.co/wMCnT53M go to 11. Now I just need to work in a miniature Stonehenge... Went to Iceland on Sat to ride bumper cars on ice! No, not the country, Vlad's rink in Van Nuys. Awesome family fun :) http://t.co/rBQXJ9IT",0
2011-12-02,33.34,52.42,NaN,NaN,,0
2011-12-03,NaN,NaN,"That was a total non sequitur btw Great Voltaire quote, arguably better than Twain. Hearing news of his own death, Voltaire replied the reports were true, only premature.",2.0,,0
2011-12-04,NaN,NaN,Am reading a great biography of Ben Franklin by Isaacson. Highly recommended.,1.0,,0
2011-12-05,34.38,53.00,NaN,NaN,,Am reading a great biography of Ben Franklin by Isaacson. Highly recommended.
2011-12-06,34.88,57.08,NaN,NaN,,0


In [8]:
clean_df_next1.head()
# weekend_corected.head(100)
# weekend_corected.loc[weekend_corected['tweet count']>=1].head(100)
# weekend_corected.loc['2011-12-01':'2011-12-30'].head(10)

NameError: name 'clean_df_next1' is not defined

In [493]:
clean_df_next2.head()

,TSLA,QQQ,tweet,tweet count,weekend corrected,weekend corrected1
2010-06-29,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,23.94,43.40,NaN,NaN,,
2010-07-02,23.63,39.16,NaN,NaN,,
2010-07-06,21.90,42.58,NaN,NaN,,


In [468]:
clean_df.loc[clean_df['TSLA'].isnull(),['weekend corrected','weekend corrected1']] = [clean_df['tweet'].shift(1),2]

ValueError: could not broadcast input array from shape (3243) into shape (587)

In [467]:
clean_df.loc[clean_df['TSLA'].isnull()].head(20)

,TSLA,QQQ,tweet,tweet count,weekend corrected,weekend corrected1
2011-12-03,NaN,NaN,That was a total non sequitur btw Great Voltai...,2.0,1.0,2.0
2011-12-04,NaN,NaN,Am reading a great biography of Ben Franklin b...,1.0,1.0,2.0
2011-12-24,NaN,NaN,The Russians are having some challenges with t...,3.0,1.0,2.0
2011-12-26,NaN,NaN,Walked around a neighborhood recently rebuilt ...,6.0,1.0,2.0
2011-12-31,NaN,NaN,Hacked my Tesla charge connector on a small is...,1.0,1.0,2.0
2012-01-01,NaN,NaN,I said *information* weapons for a reason. The...,8.0,1.0,2.0
2012-01-02,NaN,NaN,"On a lighter note, the RHCP are f* awesome. No...",1.0,1.0,2.0
2012-01-14,NaN,NaN,Somebody needs to tell Kim Jong-un that the sh...,2.0,1.0,2.0
2012-01-22,NaN,NaN,@TheOnion I heard Lars von Trier optioned the ...,1.0,1.0,2.0
2012-01-28,NaN,NaN,The lady doth protest too little. http://t.co...,2.0,1.0,2.0


In [103]:
tweets_df.head(100)

,tweet
2021-01-15,@SuperclusterHQ @w00ki33 Fallout New Texas
2021-01-15,@Breedlove22 @benmezrich Only Chuck Norris can...
2021-01-15,@Cerberu21014829 @Breedlove22 @benmezrich Good...
2021-01-15,@Breedlove22 @benmezrich The thing we call mon...
2021-01-15,Monty Python is amazing https://t.co/UJq94IWT88
...,...
2021-01-02,So proud of the Tesla team for achieving this ...
2021-01-02,"@newscientist Um, we have giant fusion reactor..."
2021-01-01,@comma_ai Tesla Full Self-Driving will work at...
2020-12-31,"@PPathole Dojo isn’t needed, but will make sel..."


In [101]:
new_tweets_df.tail(100)

,tweet
2020-09-30,@PPathole Haha true. Why do showers have such ...
2020-10-01,@MKBHD It’s next-level@FOX10Phoenix 💰💩🤣@offici...
2020-10-02,@lexfridman I agree with Lex@Mike94160775 @Tes...
2020-10-03,"@Model3teslaJ Coming soonCall of Booty, great ..."
2020-10-04,New SpaceX droneship will be called “A Shortfa...
...,...
2021-01-11,@RationalEtienne @TheBabylonBee This is an imp...
2021-01-12,@arstechnica @SciGuySpace We’re just trying to...
2021-01-13,@Erdayastronaut Detanking &amp; inspections no...
2021-01-14,"@skorusARK Prototypes are easy, volume product..."


In [90]:
new_df

,TSLA,QQQ,tweet
2010-06-29,23.94,43.40,NaN
2010-06-30,23.63,39.16,NaN
2010-07-01,21.90,42.58,NaN
2010-07-02,19.20,38.94,NaN
2010-07-06,15.98,42.60,NaN
...,...,...,...
2021-01-13,854.42,316.00,@Tesmanian_com As promised
2021-01-13,854.42,316.00,Legalize comedy
2021-01-13,854.42,316.00,@lvladimirovBG You can steal our name/logos &a...
2021-01-14,844.21,314.25,"@skorusARK Prototypes are easy, volume product..."


In [184]:
# Export csv files with stock prices
file_name="test1.csv"
output_file = Path(f"Resources/{file_name}")
clean_df.to_csv(f"{output_file}")